# NBA Games Predictor - Basic

This notebook takes all the regular season games from the NBA 2016-17 season.  Using a series of regression & supervised learning techniques, it attempts to accurately predict the outcome of basketball games.

In [1]:
import json 
import pandas as pd
import numpy as np 
import geopy.distance

#package for flattening json in pandas df
from pandas.io.json import json_normalize

In [2]:
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

## Configuration

In [3]:
use_parsed_data = True
output_parsed_data_filename = '../input/parsed_data.csv'

calc_last_5_games = True
calc_win_loss = True
calc_win_loss_write_all = True
calc_ratings = True
use_team_keys = False
use_team_hot_encoding = False
use_team_home_away_hot_encoding = False

## Load data from file system

In [4]:
#load games - json object
if use_parsed_data == True:
    games = pd.read_csv(output_parsed_data_filename)
    games = games.drop(['Unnamed: 0'], axis=1)
else:
    with open('../input/games.json') as gamesJsonRaw:
        d = json.load(gamesJsonRaw)

    games = json_normalize(d['games'])

/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
games.sample(5)

,id,location,oddsBet365Away,oddsBet365Home,scoreAway,scoreHome,scoreQuarters,season,statsAway.Ast,statsAway.Blk,...,statsHome.seasonDefensiveRating,statsHome.last5DefensiveRating,statsAway.seasonTotalPossessions,statsAway.last5TotalPossessions,statsAway.seasonTotalPoints,statsAway.last5TotalPoints,statsAway.seasonOffensiveRating,statsAway.last5OffensiveRating,statsAway.seasonDefensiveRating,statsAway.last5DefensiveRating
3357,201003030ATL,NaN,4.50,1.22,93,112,"[{'number': 1, 'scoreHome': 32, 'scoreAway': 2...",2009,23,5,...,120.15,120.15,0.0,0.0,0,0,0.00,0.00,0.00,0.00
3896,201011140NYK,NaN,2.25,1.68,104,96,"[{'number': 1, 'scoreHome': 27, 'scoreAway': 2...",2010,28,1,...,109.79,111.98,4415.8,480.6,4907,535,111.12,111.32,108.54,108.41
3258,201002170TOR,NaN,3.40,1.33,109,102,"[{'number': 1, 'scoreHome': 16, 'scoreAway': 2...",2009,22,6,...,111.13,109.09,647.2,451.2,669,478,103.37,105.94,111.25,114.80
1453,34164,Sleep Train Arena,2.25,1.68,101,116,"[{'number': 1, 'scoreHome': 27, 'scoreAway': 2...",2016,18,7,...,113.58,114.36,4377.4,467.4,5143,567,117.49,121.31,112.65,115.96
960,32137,TD Garden,3.10,1.40,102,98,"[{'number': 1, 'scoreHome': 23, 'scoreAway': 1...",2015,19,3,...,108.35,119.45,3453.4,464.0,4040,586,116.99,126.29,112.24,105.60


In [6]:
#load stadiums - json object
with open('../input/stadiums.json') as stadiumsJsonRaw:
    stadiumsJson = json.load(stadiumsJsonRaw)

stadiums = json_normalize(stadiumsJson['stadiums'])

In [7]:
stadiums.head(5)

,lat,lng,team
0,33.757183,-84.396278,ATL
1,42.366281,-71.062266,BOS
2,40.682650,-73.974689,BRO
3,35.224519,-80.841053,CHA
4,41.880589,-87.674149,CHI


   ## Data Preperation

### Record the names of all stats - to be dropped later.

In [8]:
statsColumns = pd.Series(list(games.columns.values))
statsColumns = statsColumns.loc[statsColumns.str.startswith(('statsAway', 'statsHome'), na=False)]

In [9]:
# Replace any NaN with 0 in the stats
games[statsColumns] = games[statsColumns].fillna(value=0)

### Set Valid Data Types

In [10]:
#games['season'] = games['season'].astype('float')
games['scoreAway'] = games['scoreAway'].astype('uint8')
games['scoreHome'] = games['scoreHome'].astype('uint8')

games['statsHome.FgAtt'] = games['statsHome.FgAtt'].astype('uint8')
games['statsHome.OffReb'] = games['statsHome.OffReb'].astype('uint8')
games['statsHome.Tov'] = games['statsHome.Tov'].astype('uint8')
games['statsHome.FtAtt'] = games['statsHome.FtAtt'].astype('uint8')
games['statsHome.Pts'] = games['statsHome.Pts'].astype('uint8')

games['statsAway.FgAtt'] = games['statsAway.FgAtt'].astype('uint8')
games['statsAway.OffReb'] = games['statsAway.OffReb'].astype('uint8')
games['statsAway.Tov'] = games['statsAway.Tov'].astype('uint8')
games['statsAway.FtAtt'] = games['statsAway.FtAtt'].astype('uint8')
games['statsAway.Pts'] = games['statsAway.Pts'].astype('uint8')

#scoreQuarters          object
#teamAwayCode           object
#teamHomeCode           object

### Calculate the number of games played for season

In [11]:
if (use_parsed_data == False):

    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['gamesPlayedHome', 'gamesPlayedAway'])

    # Iterate through every team
    for team in games.teamAwayCode.unique():
        season = 0

        # Iterate through each game the team is present in.
        for index, game in games[(games['teamAwayCode'] == team) | (games['teamHomeCode'] == team)].sort_values(by='season', ascending=True).iterrows():
            if season != game.season:
                gameCount = 0
                homeGameCount = 0
                season = game.season

            gameCount += 1

            # Update game count for team - whether away or home.
            if game.teamAwayCode == team:
                data.loc[index]['gamesPlayedAway'] = gameCount
            else:
                data.loc[index]['gamesPlayedHome'] = gameCount

    data['gamesPlayedAway'] = data['gamesPlayedAway'].astype('uint8')
    data['gamesPlayedHome'] = data['gamesPlayedHome'].astype('uint8')

    # Append the results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)

In [12]:
games[['teamHomeCode', 'teamAwayCode', 'season', 'gamesPlayedHome', 'gamesPlayedAway']].sample(5)

,teamHomeCode,teamAwayCode,season,gamesPlayedHome,gamesPlayedAway
8030,LAC,DET,2014,72,41
4850,DEN,SAC,2010,36,29
6819,OKL,POR,2013,4,73
1511,BOS,SAC,2016,43,43
4683,CHI,NOP,2010,11,2


## Calculate the Win/Loss % for the Home team.

In [13]:
def calc_percentage(x, y):
    return 0 if y == 0 else (x / y)

In [14]:
if (use_parsed_data == False) & (calc_win_loss == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(
        index=range(0,len(games)), 
        columns=[
            'totalGamesHome', 'totalWinsHome', 'homeGamesHome', 'homeWinsHome', 'awayGamesHome', 'awayWinsHome',
            'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome',
            'totalGamesAway', 'totalWinsAway', 'homeGamesAway', 'homeWinsAway', 'awayGamesAway', 'awayWinsAway',
            'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway'
        ]
    )

    # Iterate through each game.
    for index, game in games.iterrows():

        # Home Team
        # ---------
        
        historicGames = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamHomeCode'] == game.teamHomeCode) | (games['teamAwayCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=True)
       
        homeGamesHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode'])])        
        homeWinsHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"])])
        
        awayGamesHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode'])])        
        awayWinsHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])])

        #totalGames = len(historicGames);
        #totalWins = len(
        #    historicGames[
        #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
        #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
        #    ]
        #)
        
        totalGamesHome = homeGamesHome + awayGamesHome
        totalWinsHome = homeWinsHome + awayWinsHome
        
        percentageTotalWinHome = round(calc_percentage(totalWinsHome, totalGamesHome), 3)
        percentageHomeWinHome = round(calc_percentage(homeWinsHome, homeGamesHome), 3)
        percentageAwayWinHome = round(calc_percentage(awayWinsHome, awayGamesHome), 3)
        
        # print('HOME', game['gamesPlayedHome'], 'TG', totalGamesHome, 'W', totalWinsHome, 'HGP', homeGamesHome, 'HW', homeWinsHome, 'AGP', awayGamesHome, 'AW', awayWinsHome)
        # print('HOME', 'W%', percentageTotalWinHome, 'HW%', percentageHomeWinHome, 'AW%', percentageAwayWinHome)
        
        # Away Team
        # ---------
        
        historicGames = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=True)
       
        homeGamesAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode'])])        
        homeWinsAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode']) & (games["scoreAway"] < games["scoreHome"])])
        
        awayGamesAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode'])])        
        awayWinsAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode']) & (games["scoreAway"] > games["scoreHome"])])

        #totalGames = len(historicGames);
        #totalWins = len(
        #    historicGames[
        #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
        #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
        #    ]
        #)
        
        totalGamesAway = homeGamesAway + awayGamesAway
        totalWinsAway = homeWinsAway + awayWinsAway
        
        percentageTotalWinAway = round(calc_percentage(totalWinsAway, totalGamesAway), 3)
        percentageHomeWinAway = round(calc_percentage(homeWinsAway, homeGamesAway), 3)
        percentageAwayWinAway = round(calc_percentage(awayWinsAway, awayGamesAway), 3)
        
        # print('AWAY', game['gamesPlayedAway'], 'TG', totalGamesAway, 'W', totalWinsAway, 'HGP', homeGamesAway, 'HW', homeWinsAway, 'AGP', awayGamesAway, 'AW', awayWinsAway)
        # print('AWAY', 'W%', percentageTotalWinAway, 'HW%', percentageHomeWinAway, 'AW%', percentageAwayWinAway)
        
        # ---------    
            
        # Update the row with the history
        data.loc[index] = [
            totalGamesHome, totalWinsHome, homeGamesHome, homeWinsHome, awayGamesHome, awayWinsHome,
            percentageTotalWinHome, percentageHomeWinHome, percentageAwayWinHome,
            totalGamesAway, totalWinsAway, homeGamesAway, homeWinsAway, awayGamesAway, awayWinsAway,
            percentageTotalWinAway, percentageHomeWinAway, percentageAwayWinAway
        ]

    # Add results to the dataset
    if (calc_win_loss_write_all == True):
        games = pd.merge(games, data, left_index=True, right_index=True)
        games[['teamHomeCode', 'teamAwayCode', 'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome', 'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway']].sample(5)
    else:
        games = pd.merge(games, data[['percentageHomeWinHome', 'percentageAwayWinAway']], left_index=True, right_index=True)
        games[['teamHomeCode', 'teamAwayCode', 'percentageHomeWinHome', 'percentageAwayWinAway']].sample(5)

### Calculate the results for the last 5 games for the Home team.

In [15]:
if (use_parsed_data == False) & (calc_last_5_games == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome'])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get the last five games for the team.
        last5games = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamAwayCode'] == game.teamHomeCode) | (games['teamHomeCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=False).head(5)

        if len(last5games) == 5:
            lastGame1WinHome = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
            lastGame2WinHome = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
            lastGame3WinHome = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
            lastGame4WinHome = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
            lastGame5WinHome = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
            #print(lastGame1WinHome, lastGame2WinHome, lastGame3WinHome, lastGame4WinHome, lastGame5WinHome)

            lastGame1AtHomeHome = last5games["teamHomeCode"].iloc[0] == game.teamHomeCode
            lastGame2AtHomeHome = last5games["teamHomeCode"].iloc[1] == game.teamHomeCode
            lastGame3AtHomeHome = last5games["teamHomeCode"].iloc[2] == game.teamHomeCode
            lastGame4AtHomeHome = last5games["teamHomeCode"].iloc[3] == game.teamHomeCode
            lastGame5AtHomeHome = last5games["teamHomeCode"].iloc[4] == game.teamHomeCode
            #print(lastGame1AtHomeHome, lastGame2AtHomeHome, lastGame3AtHomeHome, lastGame4AtHomeHome, lastGame5AtHomeHome)

            # Update the row with the history
            data.loc[index] = [lastGame1WinHome, lastGame1AtHomeHome, lastGame2WinHome, lastGame2AtHomeHome, lastGame3WinHome, lastGame3AtHomeHome, lastGame4WinHome, lastGame4AtHomeHome, lastGame5WinHome, lastGame5AtHomeHome]

    # Convert types 
    data['lastGame1WinHome'] = data['lastGame1WinHome'].astype('bool')
    data['lastGame2WinHome'] = data['lastGame2WinHome'].astype('bool')
    data['lastGame3WinHome'] = data['lastGame3WinHome'].astype('bool')
    data['lastGame4WinHome'] = data['lastGame4WinHome'].astype('bool')
    data['lastGame5WinHome'] = data['lastGame5WinHome'].astype('bool')
    data['lastGame1AtHomeHome'] = data['lastGame1AtHomeHome'].astype('bool')
    data['lastGame2AtHomeHome'] = data['lastGame2AtHomeHome'].astype('bool')
    data['lastGame3AtHomeHome'] = data['lastGame3AtHomeHome'].astype('bool')
    data['lastGame4AtHomeHome'] = data['lastGame4AtHomeHome'].astype('bool')
    data['lastGame5AtHomeHome'] = data['lastGame5AtHomeHome'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)
    
    games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

### Calculate the results for the last 5 games for the Away team.

In [16]:
if (use_parsed_data == False) & (calc_last_5_games == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinAway', 'lastGame1AtHomeAway', 'lastGame2WinAway', 'lastGame2AtHomeAway', 'lastGame3WinAway', 'lastGame3AtHomeAway', 'lastGame4WinAway', 'lastGame4AtHomeAway', 'lastGame5WinAway', 'lastGame5AtHomeAway'])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get the last five games for the team.
        last5games = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=False).head(5)

        if len(last5games) == 5:
            lastGame1WinAway = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
            lastGame2WinAway = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
            lastGame3WinAway = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
            lastGame4WinAway = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
            lastGame5WinAway = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
            #print(lastGame1WinAway, lastGame2WinAway, lastGame3WinAway, lastGame4WinAway, lastGame5WinAway)

            lastGame1AtHomeAway = last5games["teamAwayCode"].iloc[0] == game.teamAwayCode
            lastGame2AtHomeAway = last5games["teamAwayCode"].iloc[1] == game.teamAwayCode
            lastGame3AtHomeAway = last5games["teamAwayCode"].iloc[2] == game.teamAwayCode
            lastGame4AtHomeAway = last5games["teamAwayCode"].iloc[3] == game.teamAwayCode
            lastGame5AtHomeAway = last5games["teamAwayCode"].iloc[4] == game.teamAwayCode
            #print(lastGame1AtHomeAway, lastGame2AtHomeAway, lastGame3AtHomeAway, lastGame4AtHomeAway, lastGame5AtHomeAway)

            # Update the row with the history
            data.loc[index] = [lastGame1WinAway, lastGame1AtHomeAway, lastGame2WinAway, lastGame2AtHomeAway, lastGame3WinAway, lastGame3AtHomeAway, lastGame4WinAway, lastGame4AtHomeAway, lastGame5WinAway, lastGame5AtHomeAway]

    # Convert types         
    data['lastGame1WinAway'] = data['lastGame1WinAway'].astype('bool')
    data['lastGame2WinAway'] = data['lastGame2WinAway'].astype('bool')
    data['lastGame3WinAway'] = data['lastGame3WinAway'].astype('bool')
    data['lastGame4WinAway'] = data['lastGame4WinAway'].astype('bool')
    data['lastGame5WinAway'] = data['lastGame5WinAway'].astype('bool')
    data['lastGame1AtHomeAway'] = data['lastGame1AtHomeAway'].astype('bool')
    data['lastGame2AtHomeAway'] = data['lastGame2AtHomeAway'].astype('bool')
    data['lastGame3AtHomeAway'] = data['lastGame3AtHomeAway'].astype('bool')
    data['lastGame4AtHomeAway'] = data['lastGame4AtHomeAway'].astype('bool')
    data['lastGame5AtHomeAway'] = data['lastGame5AtHomeAway'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)
    
    games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

### Calculate the average shooting % for the last 5 games

In [17]:
if (use_parsed_data == False) & (calc_ratings == True):
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=[
        'statsHome.seasonTotalPossessions', 'statsHome.last5TotalPossessions',
        'statsHome.seasonTotalPoints', 'statsHome.last5TotalPoints',
        'statsHome.seasonOffensiveRating', 'statsHome.last5OffensiveRating',
        'statsHome.seasonDefensiveRating', 'statsHome.last5DefensiveRating',
        'statsAway.seasonTotalPossessions', 'statsAway.last5TotalPossessions',
        'statsAway.seasonTotalPoints', 'statsAway.last5TotalPoints',
        'statsAway.seasonOffensiveRating', 'statsAway.last5OffensiveRating',
        'statsAway.seasonDefensiveRating', 'statsAway.last5DefensiveRating'
    ])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get all previous games - Home
        allgames = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamAwayCode'] == game.teamHomeCode) | (games['teamHomeCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=False)

        totalPossessionsHome = allgames['statsHome.FgAtt'].sum() - allgames['statsHome.OffReb'].sum() + allgames['statsHome.Tov'].sum() + (0.4 * allgames['statsHome.FtAtt'].sum());
        last5PossessionsHome = allgames['statsHome.FgAtt'].head(5).sum() - allgames['statsHome.OffReb'].head(5).sum() + allgames['statsHome.Tov'].head(5).sum() + (0.4 * allgames['statsHome.FtAtt'].head(5).sum());

        totalPointsScoredHome = allgames['statsHome.Pts'].sum()
        last5PointsScoredHome = allgames['statsHome.Pts'].head(5).sum()

        totalOffensiveRatingHome = round(calc_percentage(totalPointsScoredHome, totalPossessionsHome) * 100, 2)
        last5OffensiveRatingHome = round(calc_percentage(last5PointsScoredHome, last5PossessionsHome) * 100, 2)

        totalPointsAgainstHome = allgames['statsHome.PtsAgainst'].sum()
        last5PointsAgainstHome = allgames['statsHome.PtsAgainst'].head(5).sum()

        totalDefensiveRatingHome = round(calc_percentage(totalPointsAgainstHome, totalPossessionsHome) * 100, 2)
        last5DefensiveRatingHome = round(calc_percentage(last5PointsAgainstHome, last5PossessionsHome) * 100, 2)
        
        # Get all previous games - Away
        allgames = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=False)

        totalPossessionsAway = allgames['statsHome.FgAtt'].sum() - allgames['statsHome.OffReb'].sum() + allgames['statsHome.Tov'].sum() + (0.4 * allgames['statsHome.FtAtt'].sum());
        last5PossessionsAway = allgames['statsHome.FgAtt'].head(5).sum() - allgames['statsHome.OffReb'].head(5).sum() + allgames['statsHome.Tov'].head(5).sum() + (0.4 * allgames['statsHome.FtAtt'].head(5).sum());

        totalPointsScoredAway = allgames['statsHome.Pts'].sum()
        last5PointsScoredAway = allgames['statsHome.Pts'].head(5).sum()

        totalOffensiveRatingAway = round(calc_percentage(totalPointsScoredAway, totalPossessionsAway) * 100, 2)
        last5OffensiveRatingAway = round(calc_percentage(last5PointsScoredAway, last5PossessionsAway) * 100, 2)

        totalPointsAgainstAway = allgames['statsHome.PtsAgainst'].sum()
        last5PointsAgainstAway = allgames['statsHome.PtsAgainst'].head(5).sum()

        totalDefensiveRatingAway = round(calc_percentage(totalPointsAgainstAway, totalPossessionsAway) * 100, 2)
        last5DefensiveRatingAway = round(calc_percentage(last5PointsAgainstAway, last5PossessionsAway) * 100, 2)

        data.loc[index] = [
            totalPossessionsHome, last5PossessionsHome, 
            totalPointsScoredHome, last5PointsScoredHome, 
            totalOffensiveRatingHome, last5OffensiveRatingHome,
            totalDefensiveRatingHome, last5DefensiveRatingHome,
            totalPossessionsAway, last5PossessionsAway, 
            totalPointsScoredAway, last5PointsScoredAway, 
            totalOffensiveRatingAway, last5OffensiveRatingAway,
            totalDefensiveRatingAway, last5DefensiveRatingAway
        ]

        # Update the row with the history
        # data.loc[index] = [lastGame1WinAway, lastGame1AtHomeAway, lastGame2WinAway, lastGame2AtHomeAway, lastGame3WinAway, lastGame3AtHomeAway, lastGame4WinAway, lastGame4AtHomeAway, lastGame5WinAway, lastGame5AtHomeAway]

    data.sample(5)

    # Convert types
    # data['lastGame1WinAway'] = data['lastGame1WinAway'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)

    #games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)


In [18]:
# test entry for stadium distance calc

coords_1 = (37.750267, -122.202853)
coords_2 = (34.04303865743706, -118.26711416244507)

geopy.distance.vincenty(coords_1, coords_2).miles


337.6783477035334

#### Save data to CSV to save on processing

In [19]:
if use_parsed_data == False:
    games.to_csv(output_parsed_data_filename, encoding='utf-8')

## Data Cleanse
Look at data types

In [20]:
games.dtypes

id                                   object
location                             object
oddsBet365Away                      float64
oddsBet365Home                      float64
scoreAway                             uint8
scoreHome                             uint8
scoreQuarters                        object
season                                int64
statsAway.Ast                         int64
statsAway.Blk                         int64
statsAway.BlkAgainst                  int64
statsAway.DefReb                      int64
statsAway.Ejections                   int64
statsAway.Fg2PtAtt                    int64
statsAway.Fg2PtMade                   int64
statsAway.Fg2PtPct                  float64
statsAway.Fg3PtAtt                    int64
statsAway.Fg3PtMade                   int64
statsAway.Fg3PtPct                  float64
statsAway.FgAtt                       uint8
statsAway.FgMade                      int64
statsAway.FgPct                     float64
statsAway.FoulFlag1             

Transform the Time column into a datetime object.

In [21]:
games["time"] = pd.to_datetime(games["time"])

### Convert the home and away teams to one-hot encoding 

In [22]:
if use_team_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='team')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='team')

    homeTeam = homeTeam.replace({0:np.nan})
    awayTeam = awayTeam.replace({0:np.nan})

    teams = homeTeam.fillna(awayTeam).fillna(0).astype('bool')
    # teams[['team_NYK', 'team_CLE', 'team_ATL', 'team_BOS']].head(2)
    
    games = pd.concat([games, teams], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'team_NYK', 'team_CLE']].head(2)

In [23]:
if use_team_home_away_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='hometeam').astype('bool')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='awayteam').astype('bool')

    games = pd.concat([games, homeTeam, awayTeam], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'hometeam_NYK', 'hometeam_CLE', 'awayteam_NYK', 'awayteam_CLE']].head(2)

Drop the id fields for teams.

In [24]:
#if (use_team_hot_encoding == True) | (use_team_home_away_hot_encoding == True) | (use_team_keys == False):
#    games = games.drop(["teamAwayId", "teamHomeId"], axis=1)

In [25]:
games = games.drop(["teamAwayCode", "teamHomeCode"], axis=1)

In [26]:
games = games.drop(["location", "scoreQuarters", "time"], axis=1)

In [27]:
games = games.drop(statsColumns, axis=1)

In [28]:
#gamesQuarters = json_normalize(data=d['games'], record_path='scoreQuarters', meta=['id'])
#gamesQuarters.head(10)

In [29]:
games["homeWin"] = games["scoreHome"] > games["scoreAway"]

In [30]:
# Remove games scores ... not possible to predict the future, if you already know it :)
games = games.drop(["scoreAway", "scoreHome"], axis=1)

In [31]:
# Remove any games that have less than a 5 game history.
if calc_last_5_games == True:
    games = games[(pd.notnull(games['lastGame1WinHome'])) & (pd.notnull(games['lastGame1AtHomeAway']))]

In [32]:
# Fill any NaN stats with 0
games.fillna(0, inplace=True)
#games['statsHome.Ejections'].fillna(0, inplace=True)
#games['statsAway.Ejections'].fillna(0, inplace=True)

In [33]:
# Show any NaN
games[games.isnull().T.any().T]

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame1AtHomeAway,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin


In [34]:
games.sample(3)

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame1AtHomeAway,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin
3628,201004090CLE,2.04,1.80,2009,28,24,24,18,16,14,...,False,True,False,True,False,True,True,True,True,False
2860,200912210IND,2.25,1.68,2009,76,79,75,31,38,22,...,False,True,True,True,True,False,False,False,True,False
470,31647,9.50,1.07,2015,87,69,90,59,42,32,...,True,False,False,False,True,True,False,True,True,True


In [35]:
msk = np.random.rand(len(games)) < 0.8
msk

train = games[msk]
test = games[~msk]

Y_train = train["homeWin"]
X_train = train.drop(["id","homeWin"], axis=1)

#
X_test  = test.drop(["id", "homeWin"], axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((7280, 43), (7280,), (1788, 43))

In [36]:
# Reset the index
test.index = range(len(test.index))

In [37]:
X_test.sample(5)

,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,awayGamesHome,...,lastGame1WinAway,lastGame1AtHomeAway,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway
3724,3.55,1.31,2009,88,33,88,65,45,39,43,...,True,False,False,True,True,False,False,True,False,True
6428,1.71,2.20,2013,6,56,5,3,3,3,2,...,False,True,True,True,True,False,True,False,False,False
992,1.55,2.60,2015,34,56,32,17,17,10,15,...,True,False,True,False,True,False,False,True,True,True
301,6.50,1.12,2015,78,62,82,52,37,28,45,...,False,True,False,False,False,False,True,False,True,False
6844,2.65,1.54,2013,56,1,60,35,28,20,32,...,True,True,True,True,True,True,True,True,True,True


### Logistic Regression

In [38]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

69.709999999999994

In [39]:
coeff_df = pd.DataFrame(games.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
0,oddsBet365Away,0.249638
28,lastGame3AtHomeHome,0.133958
13,percentageAwayWinHome,0.125711
39,lastGame4WinAway,0.123032
36,lastGame2AtHomeAway,0.109503
11,percentageTotalWinHome,0.088235
40,lastGame4AtHomeAway,0.060227
32,lastGame5AtHomeHome,0.053950
38,lastGame3AtHomeAway,0.048964
23,lastGame1WinHome,0.048299


In [40]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_log = round((correct / total) * 100, 2)

print(correct_log, total, correct, incorrect)

66.95 1788 1197 591


### Support Vector Machines

In [41]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

96.900000000000006

In [42]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_svc = round((correct / total) * 100, 2)

print(correct_svc, total, correct, incorrect)

59.9 1788 1071 717


### k-Nearest Neighbors

In [43]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

79.090000000000003

In [44]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_knn = round((correct / total) * 100, 2)

print(correct_knn, total, correct, incorrect)

60.07 1788 1074 714


### Gaussian Naive Bayes

In [45]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

65.400000000000006

In [46]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_gaussian = round((correct / total) * 100, 2)

print(correct_gaussian, total, correct, incorrect)

66.44 1788 1188 600


### Perceptron

In [47]:
perceptron = Perceptron(max_iter = 5)
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

59.990000000000002

In [48]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_perceptron = round((correct / total) * 100, 2)

print(correct_perceptron, total, correct, incorrect)

56.38 1788 1008 780


### Linear SVC

In [49]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

60.469999999999999

In [50]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_linear_svc = round((correct / total) * 100, 2)

print(correct_linear_svc, total, correct, incorrect)

57.05 1788 1020 768


### Stochastic Gradient Descent

In [51]:
sgd = SGDClassifier(max_iter = 5)
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

60.140000000000001

In [52]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_sgd = round((correct / total) * 100, 2)

print(correct_sgd, total, correct, incorrect)

56.6 1788 1012 776


### Decision Tree

In [53]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

100.0

In [54]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_decision_tree = round((correct / total) * 100, 2)

print(correct_decision_tree, total, correct, incorrect)

58.39 1788 1044 744


### Random Forest

In [55]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

100.0

In [56]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_random_forest = round((correct / total) * 100, 2)

print(correct_random_forest, len(Y_pred), total, correct, incorrect)

65.83 1788 1788 1177 611


## Model Evaluation Results

In [57]:
models = pd.DataFrame({
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree],
    'Correct': [correct_svc, correct_knn, correct_log, 
              correct_random_forest, correct_gaussian, correct_perceptron, 
              correct_sgd, correct_linear_svc, correct_decision_tree],
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree']})
models.sort_values(by='Correct', ascending=False)

,Correct,Model,Score
2,66.95,Logistic Regression,69.71
4,66.44,Naive Bayes,65.40
3,65.83,Random Forest,100.00
1,60.07,KNN,79.09
0,59.90,Support Vector Machines,96.90
8,58.39,Decision Tree,100.00
7,57.05,Linear SVC,60.47
6,56.60,Stochastic Gradient Decent,60.14
5,56.38,Perceptron,59.99


In [58]:
result[(result['homeWin'] != result['forecast'])].sample(10)

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin,forecast
1494,201404190TOR,2.20,1.71,2013,51,54,50,25,22,12,...,True,False,False,True,True,True,True,False,False,True
14,31243,3.20,1.37,2015,56,66,53,28,27,17,...,False,False,True,True,False,False,True,True,False,True
303,34270,1.58,2.50,2016,63,81,62,40,29,21,...,False,True,False,True,True,False,True,False,False,True
1166,201303020MIL,2.85,1.44,2012,75,18,77,33,37,17,...,False,False,True,True,False,False,False,False,True,False
1532,201411130TOR,2.25,1.68,2014,21,10,19,13,11,9,...,False,True,True,False,True,False,True,False,False,True
1367,201401190TOR,4.75,1.20,2013,69,42,67,38,34,21,...,False,True,True,True,True,True,True,False,False,True
246,33973,2.15,1.74,2016,1,59,0,0,0,0,...,True,True,False,True,False,True,True,False,False,True
1447,201403210ATL,3.40,1.33,2013,62,36,66,34,32,20,...,True,False,False,True,True,True,False,False,False,True
716,201004290POR,2.00,1.83,2009,85,24,84,49,41,25,...,True,True,False,True,True,False,False,True,False,True
378,34755,2.80,1.45,2016,7,8,7,4,3,2,...,True,False,True,True,True,True,False,True,False,True


In [59]:
result[(result['id'] == 33946) | (result['id'] == 34601)]

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin,forecast


In [60]:
games[(games['id'] == 33946) | (games['id'] == 34601)]

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame1AtHomeAway,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin
